In [1]:
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
from os import listdir

In [2]:
import tensorflow as tf
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM

from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import ModelCheckpoint

physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], enable=True)

gpus = tf.config.experimental.list_physical_devices('GPU')
print(gpus)
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [3]:
import tensorflow as tf
tf.config.list_physical_devices("GPU")

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [4]:
df1 = pd.read_csv('./MovementAAL/dataset/MovementAAL_RSS_1.csv')
df2 = pd.read_csv('./MovementAAL/dataset/MovementAAL_RSS_2.csv')

In [5]:
df1.head()

,#RSS_anchor1,RSS_anchor2,RSS_anchor3,RSS_anchor4
0,-0.90476,-0.48,0.28571,0.30
1,-0.57143,-0.32,0.14286,0.30
2,-0.38095,-0.28,-0.14286,0.35
3,-0.28571,-0.20,-0.47619,0.35
4,-0.14286,-0.20,0.14286,-0.20


In [6]:
df2.head()

,#RSS_anchor1,RSS_anchor2,RSS_anchor3,RSS_anchor4
0,-0.57143,-0.20,0.71429,0.50
1,-0.76190,-0.48,0.76190,-0.25
2,-0.85714,-0.60,0.85714,0.55
3,-0.76190,-0.40,0.71429,0.60
4,-0.76190,-0.84,0.85714,0.45


In [7]:
df1.shape, df2.shape

((27, 4), (26, 4))

In [8]:
path = 'MovementAAL/dataset/MovementAAL_RSS_'
sequences = list()
for i in range(1,315):
    file_path = path + str(i) + '.csv'
    print(file_path)
    df = pd.read_csv(file_path, header=0)
    values = df.values
    sequences.append(values)


MovementAAL/dataset/MovementAAL_RSS_1.csv
MovementAAL/dataset/MovementAAL_RSS_2.csv
MovementAAL/dataset/MovementAAL_RSS_3.csv
MovementAAL/dataset/MovementAAL_RSS_4.csv
MovementAAL/dataset/MovementAAL_RSS_5.csv
MovementAAL/dataset/MovementAAL_RSS_6.csv
MovementAAL/dataset/MovementAAL_RSS_7.csv
MovementAAL/dataset/MovementAAL_RSS_8.csv
MovementAAL/dataset/MovementAAL_RSS_9.csv
MovementAAL/dataset/MovementAAL_RSS_10.csv
MovementAAL/dataset/MovementAAL_RSS_11.csv
MovementAAL/dataset/MovementAAL_RSS_12.csv
MovementAAL/dataset/MovementAAL_RSS_13.csv
MovementAAL/dataset/MovementAAL_RSS_14.csv
MovementAAL/dataset/MovementAAL_RSS_15.csv
MovementAAL/dataset/MovementAAL_RSS_16.csv
MovementAAL/dataset/MovementAAL_RSS_17.csv
MovementAAL/dataset/MovementAAL_RSS_18.csv
MovementAAL/dataset/MovementAAL_RSS_19.csv
MovementAAL/dataset/MovementAAL_RSS_20.csv
MovementAAL/dataset/MovementAAL_RSS_21.csv
MovementAAL/dataset/MovementAAL_RSS_22.csv
MovementAAL/dataset/MovementAAL_RSS_23.csv
MovementAAL/dataset/

MovementAAL/dataset/MovementAAL_RSS_259.csv
MovementAAL/dataset/MovementAAL_RSS_260.csv
MovementAAL/dataset/MovementAAL_RSS_261.csv
MovementAAL/dataset/MovementAAL_RSS_262.csv
MovementAAL/dataset/MovementAAL_RSS_263.csv
MovementAAL/dataset/MovementAAL_RSS_264.csv
MovementAAL/dataset/MovementAAL_RSS_265.csv
MovementAAL/dataset/MovementAAL_RSS_266.csv
MovementAAL/dataset/MovementAAL_RSS_267.csv
MovementAAL/dataset/MovementAAL_RSS_268.csv
MovementAAL/dataset/MovementAAL_RSS_269.csv
MovementAAL/dataset/MovementAAL_RSS_270.csv
MovementAAL/dataset/MovementAAL_RSS_271.csv
MovementAAL/dataset/MovementAAL_RSS_272.csv
MovementAAL/dataset/MovementAAL_RSS_273.csv
MovementAAL/dataset/MovementAAL_RSS_274.csv
MovementAAL/dataset/MovementAAL_RSS_275.csv
MovementAAL/dataset/MovementAAL_RSS_276.csv
MovementAAL/dataset/MovementAAL_RSS_277.csv
MovementAAL/dataset/MovementAAL_RSS_278.csv
MovementAAL/dataset/MovementAAL_RSS_279.csv
MovementAAL/dataset/MovementAAL_RSS_280.csv
MovementAAL/dataset/MovementAAL_

In [9]:
targets = pd.read_csv('MovementAAL/dataset/MovementAAL_target.csv')
targets = targets.values[:,1]

In [10]:
groups = pd.read_csv('MovementAAL/groups/MovementAAL_DatasetGroup.csv', header=0)
groups = groups.values[:,1]

In [11]:
sequences[0]

array([[-0.90476 , -0.48    ,  0.28571 ,  0.3     ],
       [-0.57143 , -0.32    ,  0.14286 ,  0.3     ],
       [-0.38095 , -0.28    , -0.14286 ,  0.35    ],
       [-0.28571 , -0.2     , -0.47619 ,  0.35    ],
       [-0.14286 , -0.2     ,  0.14286 , -0.2     ],
       [-0.14286 , -0.2     ,  0.047619,  0.      ],
       [-0.14286 , -0.16    , -0.38095 ,  0.2     ],
       [-0.14286 , -0.04    , -0.61905 , -0.2     ],
       [-0.095238, -0.08    ,  0.14286 , -0.55    ],
       [-0.047619,  0.04    , -0.095238,  0.05    ],
       [-0.19048 , -0.04    ,  0.095238,  0.4     ],
       [-0.095238, -0.04    , -0.14286 ,  0.35    ],
       [-0.33333 , -0.08    , -0.28571 , -0.2     ],
       [-0.2381  ,  0.04    ,  0.14286 ,  0.35    ],
       [ 0.      ,  0.08    ,  0.14286 ,  0.05    ],
       [-0.095238,  0.04    ,  0.095238,  0.1     ],
       [-0.14286 , -0.2     ,  0.14286 ,  0.5     ],
       [-0.19048 ,  0.04    , -0.42857 ,  0.3     ],
       [-0.14286 , -0.08    , -0.2381  ,  0.15

In [12]:
len_sequences = []
for one_seq in sequences:
    len_sequences.append(len(one_seq))
pd.Series(len_sequences).describe()

count    314.000000
mean      42.028662
std       16.185303
min       19.000000
25%       26.000000
50%       41.000000
75%       56.000000
max      129.000000
dtype: float64

In [13]:
#Padding the sequence with the values in last row to max length
to_pad = 129
new_seq = []
for one_seq in sequences:
    len_one_seq = len(one_seq)
    last_val = one_seq[-1]
    n = to_pad - len_one_seq
   
    to_concat = np.repeat(one_seq[-1], n).reshape(4, n).transpose()
    new_one_seq = np.concatenate([one_seq, to_concat])
    new_seq.append(new_one_seq)
final_seq = np.stack(new_seq)

#truncate the sequence to length 60
from tensorflow.keras.preprocessing import sequence
seq_len = 60
final_seq=sequence.pad_sequences(final_seq, maxlen=seq_len, padding='post', dtype='float', truncating='post')

In [14]:
train = [final_seq[i] for i in range(len(groups)) if (groups[i]==2)]
validation = [final_seq[i] for i in range(len(groups)) if groups[i]==1]
test = [final_seq[i] for i in range(len(groups)) if groups[i]==3]

train_target = [targets[i] for i in range(len(groups)) if (groups[i]==2)]
validation_target = [targets[i] for i in range(len(groups)) if groups[i]==1]
test_target = [targets[i] for i in range(len(groups)) if groups[i]==3]

In [15]:
train = np.array(train)
validation = np.array(validation)
test = np.array(test)

train_target = np.array(train_target)
train_target = (train_target+1)/2

validation_target = np.array(validation_target)
validation_target = (validation_target+1)/2

test_target = np.array(test_target)
test_target = (test_target+1)/2

In [22]:
model = Sequential()
model.add(LSTM(32, input_shape=(seq_len, 4)))
model.add(Dense(1, activation='sigmoid'))

In [23]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 32)                4736      
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 33        
Total params: 4,769
Trainable params: 4,769
Non-trainable params: 0
_________________________________________________________________


In [24]:
adam = Adam(learning_rate=0.001)
chk = ModelCheckpoint('best_model.pkl', monitor='val_accuracy', save_best_only=True, mode='max', verbose=2)
model.compile(loss='binary_crossentropy', optimizer=adam, metrics=['accuracy'])
model.fit(train, train_target, epochs=200, batch_size=128, callbacks=[chk], validation_data=(validation,validation_target))


Epoch 1/200
1/1 [==============================] - 1s 1s/step - loss: 0.7072 - accuracy: 0.5094 - val_loss: 0.6981 - val_accuracy: 0.5673

Epoch 00001: val_accuracy improved from -inf to 0.56731, saving model to best_model.pkl


INFO:tensorflow:Assets written to: best_model.pkl\assets


INFO:tensorflow:Assets written to: best_model.pkl\assets


Epoch 2/200
1/1 [==============================] - 0s 18ms/step - loss: 0.7032 - accuracy: 0.5377 - val_loss: 0.6948 - val_accuracy: 0.5481

Epoch 00002: val_accuracy did not improve from 0.56731
Epoch 3/200
1/1 [==============================] - 0s 18ms/step - loss: 0.6995 - accuracy: 0.5283 - val_loss: 0.6917 - val_accuracy: 0.5481

Epoch 00003: val_accuracy did not improve from 0.56731
Epoch 4/200
1/1 [==============================] - 0s 18ms/step - loss: 0.6959 - accuracy: 0.5283 - val_loss: 0.6887 - val_accuracy: 0.5577

Epoch 00004: val_accuracy did not improve from 0.56731
Epoch 5/200
1/1 [==============================] - 0s 17ms/step - loss: 0.6925 - accuracy: 0.5472 - val_loss: 0.6859 - val_accuracy: 0.5673

Epoch 00005: val_accuracy did not improve from 0.56731
Epoch 6/200
1/1 [==============================] - 0s 17ms/step - loss: 0.6892 - accuracy: 0.5849 - val_loss: 0.6831 - val_accuracy: 0.5769

Epoch 00006: val_accuracy improved from 0.56731 to 0.57692, saving model to

INFO:tensorflow:Assets written to: best_model.pkl\assets


...

Epoch 34/200
1/1 [==============================] - 0s 18ms/step - loss: 0.5632 - accuracy: 0.8113 - val_loss: 0.6243 - val_accuracy: 0.6827

Epoch 00034: val_accuracy did not improve from 0.68269
Epoch 35/200
1/1 [==============================] - 0s 18ms/step - loss: 0.5555 - accuracy: 0.8302 - val_loss: 0.6240 - val_accuracy: 0.6827

Epoch 00035: val_accuracy did not improve from 0.68269
Epoch 36/200
1/1 [==============================] - 0s 16ms/step - loss: 0.5482 - accuracy: 0.8208 - val_loss: 0.6273 - val_accuracy: 0.6635

Epoch 00036: val_accuracy did not improve from 0.68269
Epoch 37/200
1/1 [==============================] - 0s 16ms/step - loss: 0.5412 - accuracy: 0.8396 - val_loss: 0.6355 - val_accuracy: 0.6635

Epoch 00037: val_accuracy did not improve from 0.68269
Epoch 38/200
1/1 [==============================] - 0s 17ms/step - loss: 0.5341 - accuracy: 0.8396 - val_loss: 0.6515 - val_accuracy: 0.6442

Epoch 00038: val_accuracy did not improve from 0.68269
Epoch 39/200
1/


Epoch 00075: val_accuracy did not improve from 0.68269
Epoch 76/200
1/1 [==============================] - 0s 17ms/step - loss: 0.2673 - accuracy: 0.9057 - val_loss: 1.1703 - val_accuracy: 0.5865

Epoch 00076: val_accuracy did not improve from 0.68269
Epoch 77/200
1/1 [==============================] - 0s 17ms/step - loss: 0.2613 - accuracy: 0.9057 - val_loss: 1.1632 - val_accuracy: 0.5962

Epoch 00077: val_accuracy did not improve from 0.68269
Epoch 78/200
1/1 [==============================] - 0s 16ms/step - loss: 0.2563 - accuracy: 0.9057 - val_loss: 1.1562 - val_accuracy: 0.5962

Epoch 00078: val_accuracy did not improve from 0.68269
Epoch 79/200
1/1 [==============================] - 0s 19ms/step - loss: 0.2528 - accuracy: 0.9057 - val_loss: 1.1515 - val_accuracy: 0.5962

Epoch 00079: val_accuracy did not improve from 0.68269
Epoch 80/200
1/1 [==============================] - 0s 20ms/step - loss: 0.2491 - accuracy: 0.9057 - val_loss: 1.1477 - val_accuracy: 0.5962

Epoch 00080: v

1/1 [==============================] - 0s 17ms/step - loss: 0.0622 - accuracy: 0.9811 - val_loss: 1.5020 - val_accuracy: 0.5769

Epoch 00117: val_accuracy did not improve from 0.68269
Epoch 118/200
1/1 [==============================] - 0s 17ms/step - loss: 0.0508 - accuracy: 0.9906 - val_loss: 1.5456 - val_accuracy: 0.5577

Epoch 00118: val_accuracy did not improve from 0.68269
Epoch 119/200
1/1 [==============================] - 0s 20ms/step - loss: 0.0475 - accuracy: 0.9811 - val_loss: 1.5818 - val_accuracy: 0.5577

Epoch 00119: val_accuracy did not improve from 0.68269
Epoch 120/200
1/1 [==============================] - 0s 18ms/step - loss: 0.0485 - accuracy: 0.9906 - val_loss: 1.5802 - val_accuracy: 0.5577

Epoch 00120: val_accuracy did not improve from 0.68269
Epoch 121/200
1/1 [==============================] - 0s 17ms/step - loss: 0.0367 - accuracy: 0.9906 - val_loss: 1.5968 - val_accuracy: 0.5577

Epoch 00121: val_accuracy did not improve from 0.68269
Epoch 122/200
1/1 [=====

1/1 [==============================] - 0s 18ms/step - loss: 0.0498 - accuracy: 0.9906 - val_loss: 1.6600 - val_accuracy: 0.5673

Epoch 00158: val_accuracy did not improve from 0.68269
Epoch 159/200
1/1 [==============================] - 0s 17ms/step - loss: 0.0493 - accuracy: 0.9906 - val_loss: 1.6571 - val_accuracy: 0.5673

Epoch 00159: val_accuracy did not improve from 0.68269
Epoch 160/200
1/1 [==============================] - 0s 17ms/step - loss: 0.0493 - accuracy: 0.9906 - val_loss: 1.6515 - val_accuracy: 0.5673

Epoch 00160: val_accuracy did not improve from 0.68269
Epoch 161/200
1/1 [==============================] - 0s 17ms/step - loss: 0.0496 - accuracy: 0.9906 - val_loss: 1.6669 - val_accuracy: 0.5577

Epoch 00161: val_accuracy did not improve from 0.68269
Epoch 162/200
1/1 [==============================] - 0s 17ms/step - loss: 0.0500 - accuracy: 0.9906 - val_loss: 1.6757 - val_accuracy: 0.5577

Epoch 00162: val_accuracy did not improve from 0.68269
Epoch 163/200
1/1 [=====

1/1 [==============================] - 0s 18ms/step - loss: 0.0428 - accuracy: 0.9906 - val_loss: 1.7687 - val_accuracy: 0.5673

Epoch 00199: val_accuracy did not improve from 0.68269
Epoch 200/200
1/1 [==============================] - 0s 17ms/step - loss: 0.0427 - accuracy: 0.9906 - val_loss: 1.7716 - val_accuracy: 0.5673

Epoch 00200: val_accuracy did not improve from 0.68269


In [25]:
#loading the model and checking accuracy on the test data
model = load_model('best_model.pkl')

from sklearn.metrics import accuracy_score
test_preds = model.predict_classes(test)
accuracy_score(test_target, test_preds)

c:\users\administrator\pycharmprojects\envtest\venv\lib\site-packages\tensorflow\python\keras\engine\sequential.py:455: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


0.6153846153846154

In [20]:
tf.test.gpu_device_name()

'/device:GPU:0'